In [97]:
import http.client
import json

def split_with_first_numeric(av):
    if '100%' == av[:4]:
        av = av[4:]
    for i, c in enumerate(av):
        if c.isnumeric():
            return av[:i-1], av[i:]

In [108]:
conn = http.client.HTTPSConnection("api.eu.apiconnect.ibmcloud.com")
headers = {
    'x-ibm-client-id': "7ddb67df-4c7b-4fa3-ab1f-1a5c3cd2dfc8",
    'x-ibm-client-secret': "pF0lJ3cV0vO4tJ7yF5gE5aX5oK6mI7nM8eR5cM0yI2cY8fD5lQ",
    'content-type': "application/json",
    'accept': "application/json",
    'accept-language': "fi"
    }

productean_to_attrs = {}
productean_to_cat = {}
productean_to_img = {}
cat_to_producteans = {}
total = 10000
step = int(total/10)
for offset in range(0, total+1, 100):
    if offset % step == 0:
        print(int(offset/step), '/ 10...')
    payload = "{\"view\":{\"limit\":100,\"offset\":"+str(offset)+"}}"
    conn.request("POST", "/kesko-dev-rauta-api/qa/products/actions/search", payload, headers)
    res = conn.getresponse()
    data = res.read()
    data = json.loads(data.decode("utf-8")) 
    if 'results' not in data:
        continue
    data = data['results']
    for product in data:
        if 'attributes' in product and 'articleFeatures' in product['attributes']:
            atrs = product['attributes']['articleFeatures'].split('- ')
            atrs = [atr for atr in atrs if len(atr)>0]
            if not atrs or len(atrs)==0:
                continue
            atrs = list(map(lambda atr:split_with_first_numeric(atr), atrs))
            atrs_ = []
            for a in atrs:
                if a:
                    atrs_.append(a)
            atrs = atrs_
            this_ean = product['ean']
            this_cat = product['categoryId']
            productean_to_cat[this_ean] = this_cat
            if this_cat not in cat_to_producteans:
                cat_to_producteans[this_cat] = []
            cat_to_producteans[this_cat] += [this_ean]
            if 'images' in product:
                productean_to_img[this_ean] = product['images'][0]['url']
            if this_ean in productean_to_attrs:
                print('collision!', this_ean)
                break
            productean_to_attrs[this_ean] = {}
            for atr, value in atrs:
                productean_to_attrs[this_ean][atr] = value.replace('\n','')

0 / 10...
1 / 10...
2 / 10...
3 / 10...
4 / 10...
5 / 10...
6 / 10...
7 / 10...
8 / 10...
9 / 10...
10 / 10...


KeyError: 'results'

In [109]:
productean_to_attrs_ = {}
for ean, atrs in productean_to_attrs.items():
#     print(ean, ':', atrs)
    if len(atrs) != 0:
        productean_to_attrs_[ean] = atrs
productean_to_attrs = productean_to_attrs_

In [110]:
len(productean_to_attrs),len(productean_to_cat),len(productean_to_img),len(cat_to_producteans)

(853, 1046, 1037, 253)

In [112]:
for ean, atrs in list(productean_to_attrs.items())[:10]:
    print(ean, ':', atrs)

7330509284735 : {' polyesteri,': '185g/m²'}
8711148321217 : {'Korkeus noin': '55 cm.'}
7330509184332 : {' puuvilla, twill,': '370g/m²'}
7330509264973 : {'twill,': '300g/m²', '65% polyesteri, 35% puuvilla,': '65% polyesteri, 35% puuvilla,'}
6418394704123 : {'2 vuoden tuotetakuu': '2 vuoden tuotetakuu'}
88381024044 : {'kierrosluku nopea': '0-3000 min-1', 'kierrosluku hidas': '0-600 min-1', 'ottoteho': '1200 W'}
8023806024645 : {'värivaihtelu ': '2 (melko vähän)', 'vedenimukykyluokka BIII (': '10%)'}
6405422566402 : {'100 m': '100 mm'}
88381097031 : {'karan kierre ': '14', 'vakiona': '1 kpl laikkoja', 'ottoteho': '720 W', 'kierrosluku': '11000 min -1'}
4042456085701 : {'kulutusluokka': '33', '2 sauvainen': '2 sauvainen', '1clic2go pontti': '1clic2go pontti'}


In [114]:
atr_count = {}
atr_val_count = {}
for ean, atrs in list(productean_to_attrs.items()):
    for atr, val in atrs.items():
        if atr not in atr_count:
            atr_count[atr] = 0
        atr_count[atr] += 1
        if (atr, val) not in atr_val_count:
            atr_val_count[(atr, val)] = 0
        atr_val_count[(atr, val)] += 1

In [118]:
for atr, count in sorted(atr_count.items(), key=lambda tup:tup[1], reverse=True)[:10]:
    print(atr, '_:_', count)

Pituus _:_ 44
System.Collections.Generic.List _:_ 41
Korkeus noin _:_ 40
 puuvilla, twill, _:_ 38
ottoteho _:_ 34
 _:_ 27
kasvukorkeus n. _:_ 26
Paino: _:_ 26
teho _:_ 22
paino _:_ 21


In [119]:
for atrval, count in sorted(atr_val_count.items(), key=lambda tup:tup[1], reverse=True)[:10]:
    print(atrval[0],'=',atrval[1], '_:_', count)

System.Collections.Generic.List = 1[System.String] _:_ 41
 puuvilla, twill, = 370g/m² _:_ 38
10 vuoden taku = 10 vuoden takuu _:_ 17
2-komponentti kahvall = 2-komponentti kahvalla _:_ 16
2 vuoden tuotetakuu = 2 vuoden tuotetakuu _:_ 13
100 % puuvillatwill, 370 g/m = 100 % puuvillatwill, 370 g/m² _:_ 13
65% polyesteri = 65% polyesteri _:_ 11
35% puuvilla twill, 240 g/m = 35% puuvilla twill, 240 g/m² _:_ 11
 polyesteri, = 185g/m² _:_ 10
 puuvilla, = 220 g/m² _:_ 10
